In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Function to scrape data for a single day
def scrape_data_for_date(date_str):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
    chrome_options.add_argument("--window-size=1920x1080")  # Set window size
    
    # Initialize the WebDriver with the headless options
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # Construct the URL with the current date
    url = f"https://www.wunderground.com/dashboard/pws/KGAATLAN216/table/{date_str}/{date_str}/daily"
    
    # Open the URL
    driver.get(url)
    
    try:
        # Wait for the table body element to be present in the DOM
        tbody = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//html/body/app-root/app-dashboard/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/section[1]/div[1]/div/section/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table'))
        )
        
        # Extract data from the table
        l = []
        table_rows = tbody.find_elements(By.XPATH, './/tr')

        for row in table_rows:
            row_data = [cell.text for cell in row.find_elements(By.XPATH, './/td')]
            l.append(row_data)
        
        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(l)
        df['Date'] = date_str

    except Exception as e:
        print(f"An error occurred for date {date_str}: {e}")
        df = pd.DataFrame()  # Return an empty DataFrame in case of error

    finally:
        # Close the browser
        driver.quit()
    
    print(f"Day {date_str} is done")
    
    return df

# Define the start and end dates
start_date = datetime.strptime("2023-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-01-02", "%Y-%m-%d")

# Create a list of all dates to process
dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
date_strings = [date.strftime("%Y-%m-%d") for date in dates]

# Scrape data sequentially for each date
results = [scrape_data_for_date(date_str) for date_str in date_strings]
g 
# Concatenate all the DataFrames together
all_data = pd.concat(results, ignore_index=True)

print(all_data)


In [ ]:
all_data